In [2]:
from bokeh.events import ButtonClick
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CustomJS, Button, Slider, DataRange1d, LabelSet, RadioButtonGroup, Div
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.io import output_notebook, show, curdoc
import IPython
from IPython.display import display, HTML

‘Quantum physics’ is a term widely used but much less understood. It is a mathematical model first used to describe the behavior of small things in a laboratory, which exposed gaps in the preceding theory of **‘classical’** (pre-quantum) physics. Quantum theory explains this behavior and gives a more complete picture of our universe. This previously unexplained behavior can be used to perform certain computations that was previously believed impossible. This is **quantum computing.** Quantum computing distills the core concepts from quantum physics into their simplest forms, stripping away the complications of the physical world.

## Review of Classical Probability

If a fair coin is tossed, there is a 50% chance of Heads, and a 50% chance of Tails, from whatever initial state (head or tail) the system starts from. Around 500 to 1000 tosses should be enough to get reliable results.

In [4]:
data_labels = ["Heads", "Tails"]
initial_counts = [0] * 2
initial_labels = ["0.00%"] * 2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
    plot_height=500, plot_width=500,
    sizing_mode="scale_width", tools="reset",
    x_range=data_labels,
    y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
    y_axis_label="Counts"
)
plot.vbar(top='top', x='x', width=0.5, source=source, color="#BE95FF")
labels = LabelSet(
    x='x', y='top', text='top_percent', level='glyph', 
    x_offset=-20, y_offset=12, source=source, render_mode='canvas'
)
plot.add_layout(labels)

tossbtn = Button(label="Toss Coin")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=0)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

toss_callback = CustomJS(
    args=dict(source=source, p=plot, s=repeat_slider, resultsHTML=resultsHTML, results_label=results_label), 
    code="""
        for (var i = 0; i <= s.value; ++i) {
            const result = Math.floor(Math.random() * 2);
            source.data.top[result] += 1;
            resultsHTML.text += [' H ', ' T '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " tosses):";

        for (var i = 0; i < 2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """
)

slider_callback = CustomJS(
    args=dict(tossbtn=tossbtn), 
    code="""
        const repeats = cb_obj.value;
        if (repeats == 1) {
            tossbtn.label = "Toss Coin";
        } else {
            tossbtn.label = "Toss " + repeats + " Coins";
        };
    """
)

reset_callback = CustomJS(
    args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), 
    code="""
        source.data.top = [0,0];
        source.data.top_percent = ['0.00%', '0.00%'];
        source.change.emit();
        resultsHTML.text = "";
        results_label.text = "Results:";
    """
)

tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

control_panel = column(tossbtn, repeat_slider, radio_label, init_state_radio, resetbtn, results_label, resultsHTML)
layout = row(plot, control_panel)
html_repr = file_html(layout, CDN)
HTML(html_repr)

Also the probability of Heads after these two tosses is 50%, and the probability of Tails after these two tosses is also 50%.

$$ P(H) = \frac{1}{2} \times \frac{1}{2} + \frac{1}{2} \times \frac{1}{2} = \frac{1}{4} + \frac{1}{4} = \frac{1}{2} $$
$$ P(T) = \frac{1}{2} \times \frac{1}{2} + \frac{1}{2} \times \frac{1}{2} = \frac{1}{4} + \frac{1}{4} = \frac{1}{2} $$

## The Quantum Coin

Now with the understanding of the classical coin, it’s time to introduce a quantum ‘coin’, a **‘qubit’**. 

A qubit is something to play with in a lab, as they are very difficult to manipulate. Many years of scientific and technological advancements have gone into creating the qubits we have today, but the beauty of learning through quantum computing is that we can ignore the physical complications and just remember that when a qubit is measured, it will be in one of two states: 0 and 1. A quantum toss with the qubit, and measuring the state of the qubit should look similar to the classical coin experiment. But, when two tosses are done, the result looks like this (this experiment is not done on real qubit, it's presentation only):

In [5]:
data_labels = ["0", "1"]
initial_counts = [0]*2
initial_labels = ["0.00%"]*2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
    plot_height=500, plot_width=500,
    sizing_mode="scale_width", tools="reset",
    x_range=data_labels,
    y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
    y_axis_label="Counts"
)
plot.vbar(top='top', x='x', width=0.5, source=source, color="#BE95FF")
labels = LabelSet(
    x='x', y='top', text='top_percent', level='glyph',
    x_offset=-20, y_offset=12, source=source, render_mode='canvas'
)
plot.add_layout(labels)

tossbtn = Button(label="Toss Quantum Coin Twice")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=0)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

toss_callback = CustomJS(
    args=dict(
        source=source, p=plot, s=repeat_slider, resultsHTML=resultsHTML, 
        results_label=results_label, init_state_radio=init_state_radio
    ),
    code="""
        for (var i = 0; i <= s.value; ++i){
            const result = init_state_radio.active;
            source.data.top[result] += 1;
            resultsHTML.text += [' 0 ', ' 1 '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " Double Quantum Tosses):";

        for (var i = 0; i < 2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """
)

slider_callback = CustomJS(
    args=dict(tossbtn=tossbtn), 
    code="""
        const repeats = cb_obj.value;
        if (repeats == 1) {
            tossbtn.label = "Toss Quantum Coin Twice";
        } else {
            tossbtn.label = "Toss " + repeats + " Quantum Coins Twice";
        };
    """
)

reset_callback = CustomJS(
    args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), 
    code="""
        source.data.top = [0,0];
        source.data.top_percent = ['0.00%', '0.00%'];
        source.change.emit();
        resultsHTML.text = "";
        results_label.text = "Results:";
    """
)

tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

control_panel = column(tossbtn, repeat_slider, radio_label, init_state_radio, resetbtn, results_label, resultsHTML)
layout = row(plot, control_panel)
html_repr = file_html(layout, CDN)
HTML(html_repr)

This is an unexpected result. This is the same problem that physicists encountered in the early 20th century. Searching for the answer led to the development of quantum physics.

## The Quantum Model

In short, quantum theory is probability theory with negative numbers. 

But negative probabilities doesn’t make sense. So, this entity is called _amplitudes_ (denoted with $\phi$). To get around the fact that negative probabilities are impossible, and that all probabilities must add up to 1, a mathematical trick is used: square the amplitudes to calculate the probabilities. The amplitudes for single quantum coin toss look like this:

$$ \text{Starting with initial state 0}\ , $$
$$ \phi(0_0) = \sqrt{\frac{1}{2}};\ P(0_0) = \sqrt{\frac{1}{2}}^2 = \frac{1}{2} $$
$$ \phi(1_0) = \sqrt{\frac{1}{2}};\ P(1_0) = \sqrt{\frac{1}{2}}^2 = \frac{1}{2} $$

$$ \text{But, Starting with initial state 1}\ , $$
$$ \phi(0_1) = \sqrt{\frac{1}{2}};\ P(0_1) = \sqrt{\frac{1}{2}}^2 = \frac{1}{2} $$
$$ \phi(1_1) = -\sqrt{\frac{1}{2}};\ P(1_1) = (-\sqrt{\frac{1}{2}})^2 = \frac{1}{2} $$

Why are the amplitudes $\sqrt{\tfrac{1}{2}}$? Why amplitudes are negative only when starting with state 1? Well, that's mysterious apart from the fact that they produce the right result. Maybe, Max Born's noble prize research shed some light.

Overally, it produce a 50-50 chance for both result, which is expected. But, when two tosses are chained the amplitudes and probabilities looks like this:

$$ \text{Starting with initial state 0}\ , $$
$$ \phi(0) = \phi(0_0) \times \phi(0_0) + \phi(0_1) \times \phi(1_0) = \sqrt{\frac{1}{2}} \times \sqrt{\frac{1}{2}} + \sqrt{\frac{1}{2}} \times \sqrt{\frac{1}{2}} = \frac{1}{2} + \frac{1}{2};\ P(0) = \phi(0)^2 = 1 $$
$$ \phi(1) = \phi(0_0) \times \phi(0_1) + \phi(0_1) \times \phi(1_1) = \sqrt{\frac{1}{2}} \times \sqrt{\frac{1}{2}} + \sqrt{\frac{1}{2}} \times (-\sqrt{\frac{1}{2}}) = \frac{1}{2} - \frac{1}{2};\ P(1) = \phi(1)^2 = 0 $$

The amplitudes of finding the coin (qubit) in the state 1 cancel each other out, this effect is called interference. This should be simillar when started with initial state 1, where state 0 should be cancelled out.

It turns out that these interference effects can be used as advantage; these operations such as the quantum coin toss can be combined to build more efficient algorithms. These algorithms can use interference effects to make the wrong answers cancel out quickly and give a high probability of measuring the right answer. This is the idea behind quantum computing. 